# Install Packages and Download Dataset

**https://universe.roboflow.com/mouth-detection/mouth-detection-tkoln**

In [ ]:
# Roboflow for Dataset
!pip install roboflow

# YOLOV8
!pip install ultralytics

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="ueSrp8zT1NmWHzCxoYmt")
project = rf.workspace("mouth-detection").project("mouth-detection-tkoln")
version = project.version(7)
dataset = version.download("yolov8")

In [ ]:
import ultralytics

# Check GPU
ultralytics.checks()

# Fix YAML Paths

In [ ]:
import yaml
import os

def edit_yaml_paths(yaml_file_path):
    # Dataset main directory
    root_dir = os.path.dirname(yaml_file_path)

    # Load the YAML file
    with open(yaml_file_path, 'r') as file:
        data = yaml.safe_load(file)

    # Modify YAML file to include correct subset paths
    data['train'] = os.path.join(root_dir, 'train/images')
    data['val'] = os.path.join(root_dir, 'test/images') # This dataset does not have a validation subset
    data['test'] = os.path.join(root_dir, 'test/images')

    with open(yaml_file_path, 'w') as file:
        yaml.safe_dump(data, file)

    print("YAML file updated successfully!")


In [ ]:
yaml_path = '/content/Mouth-Detection-7/data.yaml'
edit_yaml_paths(yaml_path)

# Create, Train, and Validate Model

In [ ]:
from ultralytics import YOLO
import warnings
# Load trained model
weights_path = '/content/best.pt'
try:
    object_detect_model = YOLO(weights_path)
    print(f"Loaded model with best weights from {weights_path}")
except FileNotFoundError:
    warnings.warn(f"MODEL NOT FOUND AT {weights_path}, LOADING GENERIC PRETRAINED YOLOv8 MODEL INSTEAD.")
    object_detect_model = YOLO('yolov8n.pt')  # Load a normal YOLO model with pretrained weights

In [ ]:
# TRAIN, VALIDATE, AND TEST MODEL
training = object_detect_model.train(data=yaml_path, epochs=15)
validation = object_detect_model.val()
testing = object_detect_model.val(data=yaml_path, split='test')

# Zip weights folder to be downloaded

In [ ]:
import shutil
from IPython.display import FileLink


directory_to_zip = "/content/runs"
output_zip = "/content/runs.zip"

# Zip the directory
shutil.make_archive(output_zip.replace('.zip', ''), 'zip', directory_to_zip)

# Check if the zip file was created successfully
if os.path.exists(output_zip):
    print(f"Successfully created zip file: {output_zip}")
    # Provide a link to download the zip file
    display(FileLink(output_zip))
else:
    print("Failed to create zip file")
